# Una solución empresarial completa

## Ahora llevaremos nuestro proyecto del día 1 al siguiente nivel

### DESAFÍO EMPRESARIAL:

Crear un producto que genere un folleto para una empresa que se utilizará para posibles clientes, inversores y posibles reclutas.

Se nos proporcionará un nombre de empresa y su sitio web principal.

Consulte el final de este cuaderno para ver ejemplos de aplicaciones empresariales del mundo real.

Y recuerde: ¡siempre estoy disponible si tiene problemas o ideas! No dude en comunicarse conmigo.

In [ ]:
# imports
# Si esto falla, verifica que esté ejecutándose desde un entorno "activado" con (llms) en el símbolo del sistema

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [ ]:
# Inicialización y constantes and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key[:8]=='sk-proj-':
    print("La clave de API parece buena")
else:
    print("¿Puede haber un problema con tu clave API? ¡Visita el cuaderno de resolución de problemas!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [ ]:
# La clase para representar una Página Web

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado, ahora con enlaces
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "Sin título"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Título de la Web:\n{self.title}\nContenido de la Web:\n{self.text}\n\n"

In [ ]:
frog = Website("https://cursos.frogamesformacion.com")
print(frog.get_contents())
frog.links

## Primer paso: hacer que GPT-4o-mini determine qué enlaces son relevantes

### Usar una llamada a gpt-4o-mini para leer los enlaces en una página web y responder en JSON estructurado.
Debería decidir qué enlaces son relevantes y reemplazar los enlaces relativos como "/about" con "https://company.com/about".
Usaremos "one shot prompting" en las que proporcionamos un ejemplo de cómo debería responder en la solicitud.

Este es un excelente caso de uso para un LLM, porque requiere una comprensión matizada. Imagínate intentar programar esto sin LLMs analizando la página web: ¡sería muy difícil!

Nota al margen: existe una técnica más avanzada llamada "Salidas estructuradas" en la que requerimos que el modelo responda de acuerdo con una especificación. Cubrimos esta técnica en la Semana 8 durante nuestro proyecto autónomo de inteligencia artificial Agentic.

In [ ]:
link_system_prompt = "Se te proporciona una lista de enlaces que se encuentran en una página web. \
Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, \
como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.\n"
link_system_prompt += "Debes responder en JSON como en este ejemplo:"
link_system_prompt += """
{
    "links": [
        {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
        {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
    ]
}
"""

In [ ]:
print(link_system_prompt)

In [ ]:
def get_links_user_prompt(website):
    user_prompt = f"Aquí hay una lista de enlaces de la página web {website.url} - "
    user_prompt += "Por favor, decide cuáles de estos son enlaces web relevantes para un folleto sobre la empresa. Responde con la URL https completa en formato JSON. \
No incluyas Términos y Condiciones, Privacidad ni enlaces de correo electrónico.\n"
    user_prompt += "Links (puede que algunos sean links relativos):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [ ]:
print(get_links_user_prompt(frog))

In [ ]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [ ]:
anthropic = Website("https://www.anthropic.com/")
anthropic.links
anthropic.text
#ibt = Website("https://www.ibt.unam.mx")
#ibt.links

In [ ]:
get_links("https://www.anthropic.com")

In [ ]:
get_links("https://cursos.frogamesformacion.com")

## Segundo paso: ¡crea el folleto!

Reúne todos los detalles en otro mensaje para GPT4-o

In [ ]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Links encontrados:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://www.ibt.unam.mx"))

In [ ]:
system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa\
y crea un folleto breve sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
Incluye detalles sobre la cultura de la empresa, los clientes, las carreras/empleos y los cursos/packs para futuros empleos si tienes la información."

# O descomenta las líneas a continuación para obtener un folleto más humorístico: esto demuestra lo fácil que es incorporar el "tono":

# system_prompt = "Eres un asistente que analiza el contenido de varias páginas relevantes del sitio web de una empresa \
# y crea un folleto breve, divertido y gracioso sobre la empresa para posibles clientes, inversores y nuevos empleados. Responde en formato Markdown.\
#Incluye detalles sobre la cultura de la empresa, los clientes y los cursos/packs para futuros empleos si tienes la información."


In [ ]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"Estás mirando una empresa llamada: {company_name}\n"
    user_prompt += f"Aquí se encuentra el contenido de su página de inicio y otras páginas relevantes; usa esta información para crear un breve folleto de la empresa en Markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncar si tiene más de 20.000 caracteres
    return user_prompt

In [ ]:
get_brochure_user_prompt("Instituto de Biotecnologia", "https://www.ibt.unam.mx")

In [ ]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("Instituto de Biotecnologia", "https://www.ibt.unam.mx")

In [ ]:
create_brochure("Frogames Formación", "https://cursos.frogamesformacion.com")

## Por último, una pequeña mejora

Con un pequeño ajuste, podemos cambiar esto para que los resultados se transmitan desde OpenAI,
con la animación de máquina de escribir habitual


In [ ]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [50]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    #for chunk in stream:
    #       print(   chunk.choices[0].delta.content or '')
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        
        update_display(Markdown(response), display_id=display_handle.display_id)

In [51]:
stream_brochure("Instituto de Biotecnologia", "https://www.ibt.unam.mx")

Links encontrados: {'links': [{'type': 'Pagina Sobre nosotros', 'url': 'https://ibt.unam.mx/sitio/sobre-nosotros-y-nuestra-organizacion/sobre-nosotros-y-nuestra-organizacion-23'}, {'type': 'Pagina de carreras/empleos', 'url': 'https://ibt.unam.mx/estudia-con-nosotros/estudia-con-nosotros-21'}, {'type': 'Pagina de Cursos/Packs', 'url': 'https://ibt.unam.mx/sitio/facilidades-y-servicios/facilidades-y-servicios-22'}, {'type': 'Informacion institucional', 'url': 'https://ibt.unam.mx/informacion-institucional/informacion-institucional-24'}, {'type': 'Página de publicaciones', 'url': 'https://ibt.unam.mx/publicaciones/'}, {'type': 'Página de investigaciones', 'url': 'https://ibt.unam.mx/sitio/investigacion/reas-de-conocimiento-20'}]}



# Folleto del Instituto de Biotecnología - UNAM

## Sobre Nosotros

El **Instituto de Biotecnología (IBt)** de la Universidad Nacional Autónoma de México (UNAM) se destaca como una institución líder en biotecnología, tanto a nivel nacional como internacional. Con más de 40 años de historia, nuestro objetivo principal es desarrollar la biotecnología moderna mediante investigaciones de excelencia y la formación de recursos humanos altamente especializados.

### Misión
Desarrollar la biotecnología a partir de investigaciones de frontera para contribuir a la ciencia, tecnología y sociedad.

### Visión
Consolidar el liderazgo del IBt en el ámbito científico y tecnológico.

### Objetivos
- Realizar investigación original en diversas áreas biotecnológicas.
- Colaborar con el sector industrial para el desarrollo de soluciones tecnológicas.
- Educar y formar recursos humanos a través de proyectos de investigación multidisciplinarios.

## Investigación y Áreas de Especialización

El IBt abarca varias disciplinas, entre las que se incluyen:

- **Cristalografía**
- **Bioinformática Genómica**
- **Bioingeniería**
- **Biología Molecular de Eucariotes**
- **Fisiología**
- **Microbiología**
- **Toxicología**
- **Virología**

También contamos con departamentos especializados, tales como:

- Biología Molecular de Plantas
- Genética del Desarrollo y Fisiología Molecular
- Medicina Molecular y Bioprocesos

## Nuestros Clientes

Colaboramos con instituciones nacionales e internacionales, así como con el sector industrial, abordando retos en salud, agropecuarios, industriales, energéticos y medioambientales.

## Oportunidades de Carrera

El IBt ofrece un entorno dinámico para su personal. Los interesados en unirse a nuestro equipo deben estar preparados para contribuir a nuestra misión mediante investigaciones de vanguardia. Aquí, fomentamos el desarrollo profesional a través de:

- Planes de desarrollo individual para investigadores.
- Oportunidades para jóvenes talentos en áreas prioritarias.

## Formación Académica

### Programas de Estudio
Ofrecemos una amplia variedad de programas de educación superior en biotecnología, que incluyen:

- **Doctorados**
- **Maestrías**
- **Licenciaturas**
- **Cursos especializados**

### Facilidades y Servicios
El IBt cuenta con laboratorios equipados con tecnología de punta, bibliotecas especializadas y recursos para el aprendizaje práctico en diversas áreas de investigación.

## Compromiso Social

Nos comprometemos a contribuir a la sociedad mediante la divulgación del conocimiento, la promoción de la investigación fundamental y la aplicación de tecnologías biológicas que ayuden a resolver problemas sociales y medioambientales.

## Contacto

Para más información, visita nuestro sitio web o contáctanos directamente:

- **Email**: [informes@ibt.unam.mx](mailto:informes@ibt.unam.mx)
- **Sitio web**: [Instituto de Biotecnología - UNAM](https://ibt.unam.mx)

### ¡Únete a nosotros!
Descubre el potencial de la biotecnología y cómo puedes ser parte de un equipo que está cambiando el futuro.



In [ ]:
stream_brochure("HuggingFace", "https://huggingface.co")

In [ ]:
stream_brochure("Frogames Formación", "https://cursos.frogamesformacion.com")

## Aplicaciones empresariales

En este ejercicio, ampliamos el código del día 1 para realizar múltiples llamadas a LLM y generar un documento.

En términos de técnicas, este es quizás el primer ejemplo de patrones de diseño de Agentic AI, ya que combinamos múltiples llamadas a LLM. Esto se abordará más en la semana 2 y luego volveremos a Agentic AI de manera importante en la semana 8, cuando construyamos una solución Agent completamente autónoma.

En términos de aplicaciones empresariales, generar contenido de esta manera es uno de los casos de uso más comunes. Al igual que con el resumen, esto se puede aplicar a cualquier vertical empresarial. Escriba contenido de marketing, genere un tutorial de producto a partir de una especificación, cree contenido de correo electrónico personalizado y mucho más. Explore cómo puede aplicar la generación de contenido a su negocio e intente crear un prototipo de prueba de concepto.